In [44]:
import re
import pandas as pd
pd.options.display.max_columns = 9
pd.options.display.max_rows = 3
np = pd.np
np.norm = np.linalg.norm
from datetime import datetime, date
import json
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from sklearn.feature_extraction.text import TfidfVectorizer  # equivalent to TFIDFTransformer(CountVectorizer())
from django.db.models import Sum

from pacs.models import CampaignDetail, WorkingTransactions
import django
django.setup()
CampaignDetail.objects.count(), WorkingTransactions.objects.count()



(3280, 997955)

# Django
This is how you join CampaignDetail & WorkingTransactions  
and aggregate the WorkingTransactions.amount

In [45]:
qs = CampaignDetail.objects.annotate(net_amount=Sum('workingtransactions__amount')).values().all()

Convert a Django Queryset into a Pandas DataFrame

In [46]:
df = pd.DataFrame.from_records(qs)
df

,candidate_name,committee_name,committee_subtype,committee_type,...,race,total,total_spent,website
0,Katherine Schacht,Supporters of Katherine Schacht,None,CC,...,2016 Election Director Emerald People's Utilit...,1083.15,1664.61,None
...,...,...,...,...,...,...,...,...,...
3279,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN


In [47]:
df = pd.DataFrame(df[df.committee_name.astype(bool)])
df

,candidate_name,committee_name,committee_subtype,committee_type,...,race,total,total_spent,website
0,Katherine Schacht,Supporters of Katherine Schacht,None,CC,...,2016 Election Director Emerald People's Utilit...,1083.15,1664.61,None
...,...,...,...,...,...,...,...,...,...
3278,Cottage Grove Blackberry Pie Society,Cottage Grove Blackberry Pie Society,Miscellaneous,PAC,...,,180.00,171.80,None


# Pandas DataFrame.join
What if you only want positive transactions?

In [50]:
qs_pos = CampaignDetail.objects.filter(committee_name__isnull=False, workingtransactions__amount__gt=0)
qs_pos = qs_pos.annotate(pos_amount=Sum('workingtransactions__amount'))
df_pos = df.join(pd.DataFrame.from_records(qs_pos.values('pos_amount').all())['pos_amount'])
df_pos

,candidate_name,committee_name,committee_subtype,committee_type,...,total,total_spent,website,pos_amount
0,Katherine Schacht,Supporters of Katherine Schacht,None,CC,...,1083.15,1664.61,None,8104.57
...,...,...,...,...,...,...,...,...,...
3278,Cottage Grove Blackberry Pie Society,Cottage Grove Blackberry Pie Society,Miscellaneous,PAC,...,180.00,171.80,None,NaN


In [53]:
print(df.columns)

Index([u'candidate_name', u'committee_name', u'committee_subtype',
       u'committee_type', u'db_update_status', u'election', u'filer_id',
       u'grassroots', u'instate', u'net_amount', u'num_transactions', u'party',
       u'phone', u'race', u'total', u'total_spent', u'website'],
      dtype='object')


### What if I just insert a new column with the values?

In [54]:
df = pd.DataFrame.from_records(qs)
df = pd.DataFrame(df[df.committee_name.astype(bool)])
df['pos_amount'] = pd.DataFrame.from_records(qs_pos.values('pos_amount').all())['pos_amount']
df

,candidate_name,committee_name,committee_subtype,committee_type,...,total,total_spent,website,pos_amount
0,Katherine Schacht,Supporters of Katherine Schacht,None,CC,...,1083.15,1664.61,None,8104.57
...,...,...,...,...,...,...,...,...,...
3278,Cottage Grove Blackberry Pie Society,Cottage Grove Blackberry Pie Society,Miscellaneous,PAC,...,180.00,171.80,None,NaN


# Pandas indices are tricky
Did all the rows get inserted in the right place (are the indices still alligned)


In [55]:
df == df_pos

,candidate_name,committee_name,committee_subtype,committee_type,...,total,total_spent,website,pos_amount
0,True,True,False,True,...,True,True,False,True
...,...,...,...,...,...,...,...,...,...
3278,True,True,True,True,...,True,True,False,False


In [56]:
(df == df_pos).mean()

candidate_name    1.000000
                    ...   
pos_amount        0.730705
dtype: float64

# A NaN is not equal to a NaN!
Any operation involving a NaN returns a NaN  
And NaN (like None) always evaluates to False  

In [59]:
(df == df_pos).mean() + df.isnull().mean()

candidate_name    1
                 ..
pos_amount        1
dtype: float64

# Negative transaction amounts?

In [62]:
qs_neg = CampaignDetail.objects.filter(workingtransactions__amount__lt=0)
qs_neg = qs_neg.annotate(neg_amount=Sum('workingtransactions__amount'))
df = df.join(pd.DataFrame.from_records(qs_neg.values('neg_amount').all())['neg_amount'])
df

,candidate_name,committee_name,committee_subtype,committee_type,...,total_spent,website,pos_amount,neg_amount
0,Katherine Schacht,Supporters of Katherine Schacht,None,CC,...,1664.61,None,8104.57,-145.06
...,...,...,...,...,...,...,...,...,...
3278,Cottage Grove Blackberry Pie Society,Cottage Grove Blackberry Pie Society,Miscellaneous,PAC,...,171.80,None,NaN,NaN


# Directed graph of financial transactions
Are the payee_committee_ids the same as "filer_id"?

In [63]:
filer_id = set(pd.DataFrame.from_records(WorkingTransactions.objects.values(
               'filer_id').all()).dropna().values.T[0])
payee_id = set(pd.DataFrame.from_records(WorkingTransactions.objects.values(
               'contributor_payee_committee_id').all()).dropna().values.T[0])
len(payee_id.intersection(filer_id)) * 1. / len(filer_id)

0.5310975609756098

# Good enough for Government Work
53% of payee_ids were found in the filer_id of the same Table
filer_id -> payee_id

In [66]:
qs = WorkingTransactions.objects.filter(filer_id__isnull=False, 
                                        contributor_payee_committee_id__isnull=False,
                                        amount__gt=0)
df = pd.DataFrame.from_records(qs.values().all())
_, trans = df.iterrows().next()
pd.options.display.max_rows = 30
trans

addr_line1                                       52 NE Freemont St
addr_line2                                                    None
amount                                                      320.95
book_type                                      Political Committee
city                                                      Portland
contributor_payee                          Paul For County (14406)
contributor_payee_class                                       None
contributor_payee_committee_id                               14406
direction                                                      out
filed_date                                              2012-10-22
filer                                      Charlie Hales for Mayor
filer_id_id                                                  15089
purp_desc                         error on check.  voided by bank.
purpose_codes                                                 None
state                                                         